In [1]:
# imports
import os
import numpy as np
import datetime
import pandas as pd
#import BCI_analysis
from pathlib import Path
import subprocess

No module named 'cellpose'
cannot use anatomical mode, but otherwise suite2p will run normally


In [20]:

len(upload_dict['platform'])

66

In [ ]:
# generate a csv file and collect data in folders if necessary.
metadata_dir = '/home/jupyter/bucket/Metadata/' 
mat_file_dir = "/home/jupyter/bucket/Data/Calcium_imaging/multi_session_mat/"
dlc_base_dir = os.path.abspath("/home/jupyter/bucket/Data/Behavior_videos/DLC_output/Bergamo-2P-Photostim/")
bpod_path = os.path.abspath("/home/jupyter/bucket/Data/Behavior/BCI_exported/Bergamo-2P-Photostim/")
suite2p_path = os.path.abspath("/home/jupyter/bucket/Data/Calcium_imaging/suite2p/Bergamo-2P-Photostim/")
raw_imaging_path = os.path.abspath("/home/jupyter/bucket/Data/Calcium_imaging/raw/Bergamo-2P-Photostim/")

sessionwise_data_path = os.path.abspath("/home/jupyter/bucket/Data/Calcium_imaging/sessionwise_tba/")

face_rhythm_base_dir = '/home/jupyter/bucket/Data/Behavior_videos/FaceRhythm/'
motion_energy_base_dir = os.path.abspath("/home/jupyter/bucket/Data/Behavior_videos/MotionEnergy/Bergamo-2P-Photostim/")
raw_video_path = os.path.abspath("/home/jupyter/bucket/Data/Behavior_videos/raw/Bergamo-2P-Photostim/")

CO_save_path = "/home/jupyter/bucket/CodeOcean_transfer/"

sessions_with_errors = [] # sessions that are skipped collected in this list
no_cn_sessions = []
invalid_neurons = [] # with inf
blacklist = ['data_behav_BCI29_051122.mat',#only 18 trials
             'data_behav_BCI29_052422.mat', # no CN
             'data_behav_BCI30_050322.mat',# no CN
             'data_behav_BCI34_070722.mat',# no CN
             'data_behav_BCI31_051222.mat',#only 3 trials
             'data_behav_BCI35_063022.mat',# no CN
             'data_behav_BCI39_070522.mat', #no cn
             'data_behav_BCI29_042122.mat',# no video?
             # 'data_behav_BCI29_061022.mat',#next sessions CN is missing
             # 'data_behav_BCI29_050622.mat',#next sessions CN is missing
             # #'050422'
             #'data_behav_BCI29_051322_2.mat',# error during this extraction
             'data_behav_BCI29_051722.mat',# no video?
             'data_behav_BCI29_052022.mat',# no video?
             'data_behav_BCI31_051822.mat',# no video?
             'data_behav_BCI31_051922.mat',# no video?
             'data_behav_BCI31_052022.mat',# no video?
             'data_behav_BCI31_052422.mat',# no video?
             'data_behav_BCI31_052522.mat',# no video?
             'data_behav_BCI29_051322_2.mat',# no video?
             'data_behav_BCI38_071322.mat',# no video?
             'data_behav_BCI38_071422.mat',# no video?
             'data_behav_BCI38_071822.mat',# no video?
             'data_behav_BCI38_071922.mat',# no video?
            ]

mat_files_ = os.listdir(mat_file_dir)
mat_files = []
session_dates = {}
kept_files = []
files_thrown_away = []
only_csv_metadata = False
for f in mat_files_:
    if f.startswith('data_behav'):
        if f in blacklist:
            print('{} blacklisted, removed'.format(f))
            files_thrown_away.append(f)
            continue
        mat_files.append(f)
        subject = f[11:16]
        session = f[17:23]
        if subject not in session_dates.keys():
            session_dates[subject] = {}
        session_dates[subject][datetime.datetime.strptime(session,'%m%d%y')]  = [session,f]
        kept_files.append(f)
        
upload_dict = {'platform':[],
                'acq_datetime':[],
                'subject_id':[],
                's3_bucket':[],
                'modality0':[],
                'modality0.source':[],
                'modality1':[],
                'modality1.source':[],
                'modality2':[],
                'modality2.source':[]}
# upload_df = pd.DataFrame(columns = ['platform',
#                                     'acq_datetime',
#                                     'subject_id',
#                                     's3_bucket',
#                                     'modality0',
#                                     'modality0.source',
#                                     'modality1',
#                                     'modality1.source',
#                                     'modality2',
#                                     'modality2.source'])
platform = 'single-plane-ophys'
s3_bucket = 'aind-ophys-data'
df_metadata=pd.read_csv(os.path.join(metadata_dir,'Surgeries-BCI.csv'))
for mouse in session_dates.keys():
    mouse_id  = mouse[:3]+'_'+mouse[3:]
    for session_date in np.sort(list(session_dates[mouse].keys())):
        session,filename = session_dates[mouse][session_date]
        # load metadata to find subject_id
        try:
            subject_id = int(df_metadata.loc[df_metadata['ID']==mouse,'animal#'].values[0])
        except:
            subject_id = None
            print('missing subject id for {}'.format(mouse))
        
        # load exported bpod data to find acq datetime
        behavior_fname = os.path.join(bpod_path,mouse_id, f"{session}-bpod_zaber.npy")
        bpod_dict = np.load(behavior_fname,allow_pickle = True).tolist()
        
        idx = -1
        tiffheader = np.nan
        while np.abs(idx)<len(bpod_dict['scanimage_tiff_headers']):
            try:
                tiffheader = bpod_dict['scanimage_tiff_headers'][idx].tolist()[0]
                break
            except:
                idx-=1
        last_trial_time = tiffheader['movie_start_time'] + datetime.timedelta(seconds = float(tiffheader['description_first_frame']['frameTimestamps_sec']))
        
        gotit = False
        i_ = 0
        while not gotit:
            i_-=1
            try:
                last_residual_tiff_time = bpod_dict['residual_tiff_files']['scanimage_tiff_headers'][i_]['movie_start_time']+ datetime.timedelta(seconds = float(bpod_dict['residual_tiff_files']['scanimage_tiff_headers'][i_]['description_first_frame']['frameTimestamps_sec']))
                gotit = True
            except:
                pass
        session_end_time = np.max([last_trial_time,last_residual_tiff_time])
        
        acq_datetime = datetime.datetime.strftime(session_end_time, '%Y-%m-%d %H-%M-%S')
        
        # locate raw imaging data (no copy required, only folder name)
        modality0 = 'ophys'
        modality0_source = os.path.join(raw_imaging_path,mouse_id,session)
        
        
        #copy behavior stuff in a folder
        modality1 = 'trained_behavior'
        modality1_source = Path(os.path.join(CO_save_path,'{}-{}'.format(mouse_id,session),'behavior'))
        modality1_source.mkdir(parents=True, exist_ok=True)
        copy_command = 'gsutil cp {} {} '.format(behavior_fname,str(modality1_source)+'/'+f"{session}-bpod_zaber.npy")
        #reply = os.system(copy_command)
        if not only_csv_metadata:
            subprocess.run(copy_command,shell=True)
        bpod_file_names = np.unique(bpod_dict['bpod_file_names'])
        
        command_list = []
        for f in bpod_file_names:
            copy_command = 'gsutil -m cp -r {} {} '.format('/home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/'+f[:-4],
                                                     str(modality1_source)+'/')
            command_list.append(copy_command)

        bash_command = r" && ".join(command_list)
        #os.system(bash_command)
        if not only_csv_metadata:
            for bash_command in command_list:
                subprocess.run(bash_command,shell=True)

        
        #copy camera data in a single folder under side & bottom
        modality2 = 'behavior_videos'
        modality2_source = Path(os.path.join(CO_save_path,'{}-{}'.format(mouse_id,session),'behavior_videos'))
        modality2_source.mkdir(parents=True, exist_ok=True)
        modality2_source_side = Path(os.path.join(CO_save_path,'{}-{}'.format(mouse_id,session),'behavior_videos','side'))
        modality2_source_side.mkdir(parents=True, exist_ok=True)
        modality2_source_bottom = Path(os.path.join(CO_save_path,'{}-{}'.format(mouse_id,session),'behavior_videos','bottom'))
        modality2_source_bottom.mkdir(parents=True, exist_ok=True)
        
        side_folders = []
        bottom_folders = []
        
        for m in bpod_dict['behavior_movie_name_list']:
            if type(m) == np.ndarray:
                for movie_name in m:
                    if 'side' in movie_name:
                        side_folders.append(os.path.join('/home/jupyter/bucket/Data/Behavior_videos/raw/Bergamo-2P-Photostim',*movie_name.split('/')[5:-1]))
                    elif 'bottom' in movie_name:
                        bottom_folders.append(os.path.join('/home/jupyter/bucket/Data/Behavior_videos/raw/Bergamo-2P-Photostim',*movie_name.split('/')[5:-1]))
                    else:
                        wtf
        side_folders = np.unique(side_folders)
        command_list = []
        for s in side_folders:
            dest  = Path(os.path.join(modality2_source_side,s.split('/')[-1]))
            dest.mkdir(parents=True, exist_ok=True)
            copy_command = 'gsutil -m rsync {} {} '.format(s,
                                                     str(dest)+'/')
            command_list.append(copy_command)
        bottom_folders = np.unique(bottom_folders)
        for b in bottom_folders:
            dest  = Path(os.path.join(modality2_source_bottom,b.split('/')[-1]))
            dest.mkdir(parents=True, exist_ok=True)
            copy_command = 'gsutil -m rsync {} {} '.format(b,
                                                     str(dest)+'/')
            command_list.append(copy_command)

        bash_command = r" && ".join(command_list)
        #os.system(bash_command)
        if not only_csv_metadata:
            for bash_command in command_list:
                subprocess.run(bash_command,shell=True)
        #asd
        
        
        upload_dict['platform'].append(platform)
        upload_dict['acq_datetime'].append(acq_datetime)
        upload_dict['subject_id'].append(subject_id)
        upload_dict['s3_bucket'].append(s3_bucket)
        upload_dict['modality0'].append(modality0)
        upload_dict['modality0.source'].append(modality0_source)
        upload_dict['modality1'].append(modality1)
        upload_dict['modality1.source'].append(modality1_source)
        upload_dict['modality2'].append(modality2)
        upload_dict['modality2.source'].append(modality2_source)
        
        
        
    
        
            




data_behav_BCI29_042122.mat blacklisted, removed
data_behav_BCI29_051122.mat blacklisted, removed
data_behav_BCI29_051322_2.mat blacklisted, removed
data_behav_BCI29_051722.mat blacklisted, removed
data_behav_BCI29_052022.mat blacklisted, removed
data_behav_BCI29_052422.mat blacklisted, removed
data_behav_BCI30_050322.mat blacklisted, removed
data_behav_BCI31_051222.mat blacklisted, removed
data_behav_BCI31_051822.mat blacklisted, removed
data_behav_BCI31_051922.mat blacklisted, removed
data_behav_BCI31_052022.mat blacklisted, removed
data_behav_BCI31_052422.mat blacklisted, removed
data_behav_BCI31_052522.mat blacklisted, removed
data_behav_BCI34_070722.mat blacklisted, removed
data_behav_BCI35_063022.mat blacklisted, removed
data_behav_BCI38_071322.mat blacklisted, removed
data_behav_BCI38_071422.mat blacklisted, removed
data_behav_BCI38_071822.mat blacklisted, removed
data_behav_BCI38_071922.mat blacklisted, removed
data_behav_BCI39_070522.mat blacklisted, removed


Copying file:///home/jupyter/bucket/Data/Behavior/BCI_exported/Bergamo-2P-Photostim/BCI_29/041922-bpod_zaber.npy...
- [1 files][  4.6 MiB/  4.6 MiB]                                                
Operation completed over 1 objects/4.6 MiB.                                      
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220419-113651/20220419-113651.csv...
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220419-113651/__pycache__/user_settings.cpython-36.pyc...
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220419-113651/__init__.py...
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220419-113651/user_settings.py...
/ [4/4 files][  7.3 KiB/  7.3 KiB] 100% Done                                    
Operation complete

missing subject id for BCI32


Copying file:///home/jupyter/bucket/Data/Behavior/BCI_exported/Bergamo-2P-Photostim/BCI_32/061022-bpod_zaber.npy...
- [1 files][933.7 MiB/933.7 MiB]                                                
Operation completed over 1 objects/933.7 MiB.                                    
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220610-112831/20220610-112831.csv...
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220610-112831/__pycache__/user_settings.cpython-36.pyc...
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220610-112831/user_settings.py...
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220610-112831/__init__.py...
- [4/4 files][  7.8 KiB/  7.8 KiB] 100% Done                                    
Operation complete

missing subject id for BCI32


Copying file:///home/jupyter/bucket/Data/Behavior/BCI_exported/Bergamo-2P-Photostim/BCI_32/061422-bpod_zaber.npy...
- [1 files][801.0 MiB/801.0 MiB]                                                
Operation completed over 1 objects/801.0 MiB.                                    
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220614-100855/20220614-100855.csv...
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220614-100855/__init__.py...
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220614-100855/user_settings.py...
Copying file:///home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/20220614-100855/__pycache__/user_settings.cpython-36.pyc...
- [4/4 files][239.0 KiB/239.0 KiB] 100% Done                                    
Operation complete

In [23]:
# create csv file
output_df = pd.DataFrame.from_dict(upload_dict)
output_df.to_csv(os.path.join(CO_save_path,'uplpoad_job.csv'))

In [22]:

output_df

,platform,acq_datetime,subject_id,s3_bucket,modality0,modality0.source,modality1,modality1.source,modality2,modality2.source
0,single-plane-ophys,2022-04-19 12-05-37,608551.0,aind-ophys-data,ophys,/home/jupyter/bucket/Data/Calcium_imaging/raw/...,trained_behavior,/home/jupyter/bucket/CodeOcean_transfer/BCI_29...,behavior_videos,/home/jupyter/bucket/CodeOcean_transfer/BCI_29...
1,single-plane-ophys,2022-04-20 15-53-43,608551.0,aind-ophys-data,ophys,/home/jupyter/bucket/Data/Calcium_imaging/raw/...,trained_behavior,/home/jupyter/bucket/CodeOcean_transfer/BCI_29...,behavior_videos,/home/jupyter/bucket/CodeOcean_transfer/BCI_29...
2,single-plane-ophys,2022-04-22 12-39-41,608551.0,aind-ophys-data,ophys,/home/jupyter/bucket/Data/Calcium_imaging/raw/...,trained_behavior,/home/jupyter/bucket/CodeOcean_transfer/BCI_29...,behavior_videos,/home/jupyter/bucket/CodeOcean_transfer/BCI_29...
3,single-plane-ophys,2022-04-25 12-39-50,608551.0,aind-ophys-data,ophys,/home/jupyter/bucket/Data/Calcium_imaging/raw/...,trained_behavior,/home/jupyter/bucket/CodeOcean_transfer/BCI_29...,behavior_videos,/home/jupyter/bucket/CodeOcean_transfer/BCI_29...
4,single-plane-ophys,2022-04-27 10-56-53,608551.0,aind-ophys-data,ophys,/home/jupyter/bucket/Data/Calcium_imaging/raw/...,trained_behavior,/home/jupyter/bucket/CodeOcean_transfer/BCI_29...,behavior_videos,/home/jupyter/bucket/CodeOcean_transfer/BCI_29...
...,...,...,...,...,...,...,...,...,...,...
61,single-plane-ophys,2022-07-08 18-22-58,613725.0,aind-ophys-data,ophys,/home/jupyter/bucket/Data/Calcium_imaging/raw/...,trained_behavior,/home/jupyter/bucket/CodeOcean_transfer/BCI_37...,behavior_videos,/home/jupyter/bucket/CodeOcean_transfer/BCI_37...
62,single-plane-ophys,2022-07-06 10-10-51,621023.0,aind-ophys-data,ophys,/home/jupyter/bucket/Data/Calcium_imaging/raw/...,trained_behavior,/home/jupyter/bucket/CodeOcean_transfer/BCI_39...,behavior_videos,/home/jupyter/bucket/CodeOcean_transfer/BCI_39...
63,single-plane-ophys,2022-07-07 09-43-59,621023.0,aind-ophys-data,ophys,/home/jupyter/bucket/Data/Calcium_imaging/raw/...,trained_behavior,/home/jupyter/bucket/CodeOcean_transfer/BCI_39...,behavior_videos,/home/jupyter/bucket/CodeOcean_transfer/BCI_39...
64,single-plane-ophys,2022-07-12 13-13-43,621023.0,aind-ophys-data,ophys,/home/jupyter/bucket/Data/Calcium_imaging/raw/...,trained_behavior,/home/jupyter/bucket/CodeOcean_transfer/BCI_39...,behavior_videos,/home/jupyter/bucket/CodeOcean_transfer/BCI_39...
